# Chap06 - 텍스트 2: 단어 벡터, 고급 RNN, 임베딩 시각화

> [5장](http://excelsior-cjh.tistory.com/154)에서 살펴본 텍스트 시퀀스를 좀 더 깊이 알아보며, **word2vec**이라는 비지도학습 방법을 사용하여 단어 벡터를 학습하는 방법과 텐서보드를 사용해서 임베딩을 시각화 하는 방법에 대해 알아보자. 그리고 RNN의 업그레이드 버전인 **GRU**에 대해서 알아보자.

## 6.1 단어 임베딩 소개

[5.3.2](http://excelsior-cjh.tistory.com/154)에서 텐서플로(TensorFlow)를 이용해 텍스트 시퀀스를 다루는 방법을 알아 보았다. 단어 ID를 저차원의 Dense vector로의 매핑을 통해 단어 벡터를 학습시켰다. 이러한 처리가 필요한 이유는 RNN의 입력으로 넣어 주기 위해서였다.

> TensorFlow is an open source software library for high performance numerical computation.

위의 문장을 [5.3.2](http://excelsior-cjh.tistory.com/154)에서처럼, 각 단어를 ID로 표현 한다면 'tensorflow'는 정수 2049에, 'source'라는 단어는 17, 'performance'는 0으로 매핑할 수 있다.

하지만, 이러한 방법은 몇 가지 문제점이 있다. 

- 단어의 의미를 잃어버리게 되고, 단어 사이의 의미론적 근접성(semantic proximity)과 관련 정보를 놓치게 된다. 예를 들어, 위의 문장에서 'high'와 'software'는 서로 관련이 없지만, 이러한 정보는 반영되지 않는다.
- 단어의 수가 엄청 많을 경우 단어당 ID개수 또한 많아지게 되므로 단어의 벡터 표현이 희소(sparse)해져 학습이 어려워진다.

이러한 문제를 해결하기 위한 방법 중 하나는 비지도학습(Unsupervised Learning)인 word2vec을 이용하는 것이다. 이 방법의 핵심은 **분포 가설(Distributional Hypothesis)**이며, 언어학자 존 루퍼트 퍼스(John Rupert Firth)가 한 유명한 말로 설명할 수 있다.

> "You shall know a word by the company it keeps."- "단어는 포함된 문맥 속에서 이해할 수 있다."

즉, 비슷한 맥락에서 함께 나타나는 경향이 있는 단어들은 비슷한 의미를 가지는 경향이 있다.

## 6.2 word2vec

**word2vec**은 2013년에 [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1310.4546.pdf)(Mikolov et al.) 논문에서 등장한 비지도학습의 원드 임베딩 방법이다. word2vec에는 아래의 그림과 같이 두가지 구조(architecture)가 있는데, 이번 구현은 **skip-gram**을 이용해 단어의 문맥을 예측하는 모델을 학습한다. word2vec의 이론에 대해 자세한 설명은 [ratsgo's blog](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/03/30/word2vec/)를 참고하면 된다.

![](./images/w2v.png)

word2vec의 **skip-gram** 모델은 아래의 그림에서 볼 수 있듯이, 중심단어에서 윈도우 크기(Window size)만큼의 주변 단어들을 예측하는 모델이다.

![](./images/skip-gram.png)

word2vec 모델은 학습시간을 줄이기 위해 트릭을 쓰는데, 바로 **네거티브 샘플링(negative sampling)** 이다. 네거티브 샘플링은 위의 그림에서 처럼 'Training Sample'과 같은 단어 쌍들에 포함되어 있지 않는 **가짜** 단어 쌍들을 만들어 낸다. 즉, 지정한 윈도우 사이즈 내에서 포함되지 않는 단어들을 포함시켜 단어 쌍들을 만들어 내는것이다. 예를 들어 위의 그림에서 두번째 줄에서 윈도우 사이즈 내에 포함되지 않는 `lazy`라는 단어를 뽑아 `(quick, lazy)`라는 단어쌍을 만드는 것이다.

이렇게 '진짜'와 '가짜' 단어를 섞어 (학습 데이터, 타겟) 데이터를 만들고 이것을 구분할 수 있는 이진 분류기(binary classifier)를 학습시킨다. 이 분류기에서 학습된 가중치($\mathrm{W}$)벡터가 바로 **워드 임베딩**이다. (아래 그림 출처: [Lil'Log](https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html))

![](./images/word2vec-skip-gram.png)

### 6.2.1 Skip-Gram 구현

텐서플로(TensorFlow)를 이용해 기본적인 word2vec 모델을 구현해보자. 여기서는 [5.3.1 텍스트 시퀀스](http://excelsior-cjh.tistory.com/154)에서와 마찬가지로 '홀수'와 '짝수'로 이루어진 두 종류의 '문장'인 가상의 데이터를 생성해 word2vec을 구현해 보도록 하겠다. 기회가 된다면, 영어 및 한글의 실제 데이터를 가지고 word2vec을 구현하는 것을 추후에 포스팅하도록 하겠다. 

In [1]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from pprint import pprint

In [24]:
####################
# Hyper Parameters #
####################
batch_size = 64
embedding_dimension = 5
negative_samples = 8
LOG_DIR = './logs/word2vec_intro'


digit_to_word_map = {1: "One", 2: "Two", 3: "Three", 4: "Four", 5: "Five",
                     6: "Six", 7: "Seven", 8: "Eight", 9: "Nine"}
sentences = []

# 홀수 시퀀스/짝수 시퀀스 두 종류의 문장을 생성
for i in range(10000):
    rand_odd_ints = np.random.choice(range(1, 10, 2), size=3)
    sentences.append(" ".join([digit_to_word_map[r] for r in rand_odd_ints]))
    rand_even_ints = np.random.choice(range(2, 10, 2), size=3)
    sentences.append(" ".join([digit_to_word_map[r] for r in rand_even_ints]))

# 생성된 문장 확인
pprint(sentences[0: 10])

['Nine Five Seven',
 'Four Eight Two',
 'Three Five Nine',
 'Eight Two Four',
 'Seven Five Three',
 'Two Four Two',
 'One Nine One',
 'Eight Two Four',
 'Three Seven Five',
 'Two Eight Eight']


In [4]:
# 단어를 인덱스에 매핑
word2index_map = {}
index = 0
for sent in sentences:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index+=1

index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

print('word2index_map >>>', word2index_map)
print('index2word_map >>>', index2word_map)
print('vocabulary_size >>>', vocabulary_size)

word2index_map >>> {'five': 0, 'nine': 1, 'two': 2, 'eight': 3, 'seven': 4, 'one': 5, 'four': 6, 'six': 7, 'three': 8}
index2word_map >>> {0: 'five', 1: 'nine', 2: 'two', 3: 'eight', 4: 'seven', 5: 'one', 6: 'four', 7: 'six', 8: 'three'}
vocabulary_size >>> 9


이제 필요한 데이터를 생성했으니, word2vec skip-gram 모델을 만들어보자. 이번 구현 예제에서는 윈도우 사이즈를 1로 설정했다.

In [5]:
# Skip-Gram 쌍(pair) 생성 (Window=1)
skip_gram_pairs = []
for sent in sentences:
    tokenized_sent = sent.lower().split()
    for i in range(1, len(tokenized_sent)-1):
        word_context_pair = [[word2index_map[tokenized_sent[i-1]],
                              word2index_map[tokenized_sent[i+1]]],
                             word2index_map[tokenized_sent[i]]]
        skip_gram_pairs.append([word_context_pair[1],
                                word_context_pair[0][0]])
        skip_gram_pairs.append([word_context_pair[1],
                                word_context_pair[0][1]])

In [6]:
print(skip_gram_pairs[0:10])

[[1, 0], [1, 1], [3, 2], [3, 3], [5, 4], [5, 5], [7, 6], [7, 7], [0, 1], [0, 1]]


위의 `skip_gram_pairs`는 리스트안에 리스트 형태로 `(데이터, 타겟)` 형태로 skip-gram 쌍을 구현한 것을 확인할 수 있다. 이것을 `batch_size`만큼 가져오는 것을 아래와 같이 `get_skipgram_batch`함수로 구현하였다.

In [7]:
def get_skipgram_batch(batch_size):
    instance_indices = list(range(len(skip_gram_pairs)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [skip_gram_pairs[i][0] for i in batch]
    y = [[skip_gram_pairs[i][1]] for i in batch]
    return x, y

In [8]:
# mini-batch example
x_batch, y_batch = get_skipgram_batch(8)
print('x_batch :', x_batch)
print([index2word_map[word] for word in x_batch])
print('-'*30)
print('y_batch :', y_batch)
print([index2word_map[word[0]] for word in y_batch])

x_batch : [2, 3, 2, 2, 1, 7, 2, 6]
['two', 'eight', 'two', 'two', 'nine', 'six', 'two', 'four']
------------------------------
y_batch : [[2], [6], [3], [6], [0], [6], [6], [7]]
['two', 'four', 'eight', 'four', 'five', 'four', 'four', 'six']


이제 입력과 타깃(레이블)에 사용할 텐서플로의 플레이스홀더를 생성해준다.

In [9]:
# 입력 데이터와 레이블
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

### 6.2.2. 텐서플로에서의 임베딩

텐서플로의 [`tf.nn.embedding_lookup()`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) 함수를 사용해 임베딩한다. 워드 임베딩은 단어를 벡터로 매핑하는 룩업 테이블(look-up table)로 볼 수 있다.

In [10]:
with tf.name_scope('embeddings'):
    embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_dimension],
                              -1.0, 1.0), name='embedding')
    # This is essentialy a lookup table
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

### 6.2.3 Noise-Contrastive Estimation(NCE) 손실함수

위에서 skip-gram 모델을 만들 때, '진짜' (단어, 타겟)쌍 뿐만 아니라 '가짜' 노이즈(noise) 쌍도 같이 만들어줘서 학습할때 사용한다고 했다. 따라서, 진짜와 노이즈를 구분할 수 있도록 학습을 시켜야 한다. 

텐서플로에서는 이러한 학습을 할 수 있도록 Noise-Constratrive Estimation(NCE)라는 손실함수를 제공한다. [`tf.nn.nce_loss()`](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss)를 사용하면 세션 스코프 내에서 손실을 계산할 때 노이즈(가짜) 표본을 자동으로 만들어 준다. NCE의 식은 다음과 같다.

$$
\mathcal{L}_\theta = - [ \log \frac{\exp({v'_w}^{\top}{v_{w_I}})}{\exp({v'_w}^{\top}{v_{w_I}}) + Nq(\tilde{w})} +  \sum_{\substack{i=1 \\ \tilde{w}_i \sim Q}}^N \log \frac{Nq(\tilde{w}_i)}{\exp({v'_w}^{\top}{v_{w_I}}) + Nq(\tilde{w}_i)}]
$$

In [11]:
# Create variables for the NCE loss
nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_dimension],
                            stddev=1.0 / math.sqrt(embedding_dimension)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# NCE loss
loss = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, inputs=embed, labels=train_labels,
                   num_sampled=negative_samples, num_classes=vocabulary_size))
tf.summary.scalar('NCE_loss', loss)

<tf.Tensor 'NCE_loss:0' shape=() dtype=string>

### 6.2.4 학습률 감소

경사 하강법(Gradient Descent) 최적화는 손실함수를 최소화하는 방향으로 조금씩 이동하면서 가중치를 조정한다. `learning rate`인 $\eta$는 하이퍼파라미터로서 이동하는 크기를 결정하는 매개변수이다. **학습률 감소(learning rate decay)** 기법은 학습이 진행될수록 해당 모델의 손실값이 최저 지점에 수렴하게 되므로 학습률의 크기를 점차 감소시켜, 경사 하강법 최적화 프로세스가 **안정화** 되도록한다.

[`tf.train.exponential_decay()`](https://www.tensorflow.org/api_docs/python/tf/train/exponential_decay)은 학습률을 지수적으로 감소시킨다. 이를 식으로 나타내면 다음과 같다.

$$
\eta \leftarrow \eta e^{-kt}
$$

위의 식에서 $\eta$는 학습률, $t$는 decay_step, $k$는 decay_rate이다. 아래 코드는 1,000 단계마다 학습률을 감소시켜주는 코드이다.

In [12]:
# learning_rate decay
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(learning_rate=0.1,
                                           global_step=global_step,
                                           decay_steps=1000,
                                           decay_rate=0.95,
                                           staircase=True)
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

### 6.2.5 텐서보드를 사용하여 학습하고 시각화하기

`tf.Session()`내에서 그래프를 학습할 때 텐서보드에 시각화 해줄 부분을 코드로 작성하면 시각화가 가능하다. 2016년 하반기부터 텐서보드에서 고차원 데이터의 임베딩을 시각화하는 기능을 제공하기 시작했다. 텐서보드에 임베딩을 시각화 해주기 위해서는 다음과 같이 해주면 된다.

1. `tsv` 형식의 메타데이터 파일을 생성한다. 이 파일은 임베딩 벡터를 연관 레이블이나 이미지와 연결한다. 이번 word2vec 예제에서는 각 임베딩 벡터는 해당 단어와 매핑되어 있다.
2. 텐서보드에 임베딩 변수를 지정하고 이 변수를 메타데이터 파일에 연결한다.
3. `tf.Session()`을 닫기 전에 워드 임베딩 벡터를 단위 길이로 정규화 해준다.

In [16]:
# 모든 요약 연산을 병합
merged = tf.summary.merge_all()

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR,
                                         graph=tf.get_default_graph())
    saver = tf.train.Saver()
    
    with open(os.path.join(LOG_DIR, 'metadata.tsv'), "w") as metadata:
        metadata.write('Name\tClass\n')
        for k, v in index2word_map.items():
            metadata.write('%s\t%d\n' % (v, k))
            
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embeddings.name
    # 임베딩을 메타데이터 파일과 연결
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')
    projector.visualize_embeddings(train_writer, config)
    
    tf.global_variables_initializer().run()
    
    for step in range(1000):
        x_batch, y_batch = get_skipgram_batch(batch_size)
        summary, _ = sess.run([merged, train_step], 
                              feed_dict={train_inputs: x_batch,
                                         train_labels: y_batch})
        train_writer.add_summary(summary, step)
        
        if step % 100 == 0:
            saver.save(sess, os.path.join(LOG_DIR, 'w2v_model.ckpt'), step)
            loss_value = sess.run(loss, feed_dict={train_inputs: x_batch,
                                                   train_labels: y_batch})
            print("Loss at %d: %.5f" % (step, loss_value))
            
    # 사용 전 임베딩 정규화
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    normalized_embeddings_matrix = sess.run(normalized_embeddings)

Loss at 0: 6.80573
Loss at 100: 3.30649
Loss at 200: 2.86252
Loss at 300: 2.61449
Loss at 400: 2.53068
Loss at 500: 2.55754
Loss at 600: 2.50839
Loss at 700: 2.48901
Loss at 800: 2.54704
Loss at 900: 2.52186


### 6.2.6 임베딩 확인해보기

학습이 끝났으니, 단어벡터들을 확인해 보도록 하자. 아래의 코드는 단어 `one`을 선택해 `one`과 코사인 유사도가 가까운 순으로 내림차순하여 정렬한 것이다.

In [21]:
ref_word = normalized_embeddings_matrix[word2index_map['one']]

cosine_dists = np.dot(normalized_embeddings_matrix, ref_word)
ff = np.argsort(cosine_dists)[::-1][1:10]
for f in ff:
    print('word: %s\tsim: %.5f' % (index2word_map[f], cosine_dists[f]))

word: five	sim: 0.95848
word: seven	sim: 0.93927
word: nine	sim: 0.85144
word: three	sim: 0.71123
word: eight	sim: 0.03035
word: two	sim: -0.09810
word: four	sim: -0.11571
word: six	sim: -0.20095


위의 결과에서 확인할 수 있듯이, `five, seven, ninge, three`와 같이 홀수를 나타내는 단어들이 `one`과 가깝고 `eight, two, four, six`의 짝수를 나타내는 단어들은 `one`과 가깝지 않음을 알 수 있다.

이번에는 터미널(또는 cmd 창)에서 아래의 명령어를 통해 텐서보드(TensorBoard)를 실행시키고, **PROJECTOR** 탭에서 임베딩된 단어 벡터를 확인해보자.

```bash
#LOG_DIR = ./logs/word2vec_intro
tensorboard --logdir=LOG_DIR  # logs 디렉터리 경로
```